In [1]:
%load_ext autoreload
%autoreload 2
import os, pickle, csv, itertools, shutil, random, re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import torchvision, torch
from torchvision import transforms
from omegaconf import OmegaConf as oc
from sklearn import datasets
# %matplotlib inline
# %config InlineBackend.figure_format='retina'

import utils.utils as utils
import evals.embed_evals as evals
import utils.plot_data as plot
import utils.gen_triplets as gen
def normalize(data):return (data-np.min(data)) / (np.max(data)-np.min(data))
import warnings
warnings.filterwarnings("ignore")
from scipy import stats
def get_ci(samples, confidence=0.95): return 2 * stats.sem(samples) * stats.t.ppf((1 + confidence) / 2., len(samples)-1)


In [6]:
out_dir = "link/wv_3d_linear0"
data = "wv_3d_linear0"

eval_cols = ['NINO_ds_acc',  'NIFO_ds_acc', 'NI_h2h_d50', 'NO_h2h_d50', 'NI_h2h_d512', 'NO_h2h_d512', 'test_triplet_acc','test_clf_acc', 'test_1nn_acc', 'rNINO_ds_acc',]
config_cols = ["seed", "weights", "filtered", "embed_dim", "lamda", "wandb_group", "wandb_project"]

## aligns

In [3]:
df = utils.get_wandb_runs(data + "_aligns")
df = utils.strip_df(df, eval_cols, config_cols)
df["align"] = [utils.find_float(x)[0] for x in df["wandb_group"]]

In [4]:
np.unique(df["filtered"])

array([False,  True])

In [5]:
filtered_df = df[df["filtered"]==True]
unfiltered_df = df[df["filtered"]==False]
filtered_50 = filtered_df[filtered_df["embed_dim"]==50]
filtered_512 = filtered_df[filtered_df["embed_dim"]==512]
unfiltered_50 = unfiltered_df[unfiltered_df["embed_dim"]==50]
unfiltered_512 = unfiltered_df[unfiltered_df["embed_dim"]==512]
print(len(filtered_50))
print(len(filtered_512))
print(len(unfiltered_50))
print(len(unfiltered_512))

40
40
40
40


In [7]:
filtered_50.to_csv(out_dir + "/d=50/" + data + "_filtered_d50.csv",index=False)
filtered_512.to_csv(out_dir + "/d=512/" + data + "_filtered_d512.csv",index=False)
unfiltered_50.to_csv(out_dir + "/d=50/" + data + "_unfiltered_d50.csv",index=False)
unfiltered_512.to_csv(out_dir + "/d=512/" + data + "_unfiltered_d512.csv",index=False)

## noisy

In [8]:
df = utils.get_wandb_runs(data + "_noisy")
df = utils.strip_df(df, eval_cols, config_cols)
df["align"] = [utils.find_float(x)[0] for x in df["wandb_group"]]
df["noise"] = [utils.find_float(x)[1] for x in df["wandb_group"]]

In [9]:
np.unique(df["filtered"])

array([False,  True])

In [10]:
filtered_df = df[df["filtered"]==True]
unfiltered_df = df[df["filtered"]==False]
filtered_50 = filtered_df[filtered_df["embed_dim"]==50]
filtered_512 = filtered_df[filtered_df["embed_dim"]==512]
unfiltered_50 = unfiltered_df[unfiltered_df["embed_dim"]==50]
unfiltered_512 = unfiltered_df[unfiltered_df["embed_dim"]==512]
print(len(filtered_50))
print(len(filtered_512))
print(len(unfiltered_50))
print(len(unfiltered_512))

96
96
96
96


In [11]:
filtered_50.to_csv(out_dir + "/d=50/" + data + "_noisy" + "_filtered_d50.csv",index=False)
filtered_512.to_csv(out_dir + "/d=512/" + data + "_noisy" + "_filtered_d512.csv",index=False)
unfiltered_50.to_csv(out_dir + "/d=50/" + data + "_noisy" + "_unfiltered_d50.csv",index=False)
unfiltered_512.to_csv(out_dir + "/d=512/" + data + "_noisy" + "_unfiltered_d512.csv",index=False)

## num

In [12]:
eval_cols_num = eval_cols.copy()
eval_cols_num.remove("test_clf_acc")

In [13]:
df = utils.get_wandb_runs(data + "_num")
df = utils.strip_df(df, eval_cols_num, config_cols)
df["align"] = [utils.find_float(x)[0] for x in df["wandb_group"]]
df["num"] = [utils.find_float(x)[1] for x in df["wandb_group"]]
df["test_clf_acc"] = [0]*len(df)

In [14]:
filtered_df = df[df["filtered"]==True]
unfiltered_df = df[df["filtered"]==False]
filtered_50 = filtered_df[filtered_df["embed_dim"]==50]
filtered_512 = filtered_df[filtered_df["embed_dim"]==512]
unfiltered_50 = unfiltered_df[unfiltered_df["embed_dim"]==50]
unfiltered_512 = unfiltered_df[unfiltered_df["embed_dim"]==512]
print(len(filtered_50))
print(len(filtered_512))
print(len(unfiltered_50))
print(len(unfiltered_512))

4
4
84
84


In [15]:
filtered_50.to_csv(out_dir + "/d=50/" + data + "_num" + "_filtered_d50.csv",index=False)
filtered_512.to_csv(out_dir + "/d=512/" + data + "_num" + "_filtered_d512.csv",index=False)
unfiltered_50.to_csv(out_dir + "/d=50/" + data + "_num" + "_unfiltered_d50.csv",index=False)
unfiltered_512.to_csv(out_dir + "/d=512/" + data + "_num" + "_unfiltered_d512.csv",index=False)

## RESN

In [22]:
df = pd.read_csv("evals/RESN_evals_d50.csv")
df["align"] = [utils.find_float(x)[0] for x in df["group"]]
df = df.rename({"group":"wandb_group"},axis=1)
df["name"] = ["RESN"]*len(df)

In [23]:
for col in config_cols+eval_cols:
    if col not in df.columns: df[col] = [None]*len(df)

In [28]:
df.to_csv(out_dir + "/d=50/" + "RESN.csv",index=False)

In [29]:
df = pd.read_csv("evals/RESN_evals_d512.csv")
df["align"] = [utils.find_float(x)[0] for x in df["group"]]
df = df.rename({"group":"wandb_group"},axis=1)
df["name"] = ["RESN"]*len(df)

In [30]:
for col in config_cols+eval_cols:
    if col not in df.columns: df[col] = [None]*len(df)

In [31]:
df.to_csv(out_dir + "/d=512/" + "RESN.csv",index=False)

## scratch

In [122]:
df[df["align"]==1].groupby(["name"]).mean()

,seed,filtered,embed_dim,lamda,align,NINO_ds_acc,NIFO_ds_acc,NI_h2h,NO_h2h,test_triplet_acc,test_clf_acc,test_1nn_acc,rNINO_ds_acc
name,,,,,,,,,,,,,
MTL0.2,1.0,1.0,512.0,0.2,1.0,0.986600,0.986600,0.862786,0.911083,0.972380,0.986600,0.989112,0.986600
MTL0.5,1.0,1.0,512.0,0.5,1.0,0.989950,0.979899,0.864740,0.898241,0.975604,0.979899,0.987437,0.985762
MTL0.8,1.0,1.0,512.0,0.8,1.0,0.989950,0.988275,0.849386,0.922390,0.974276,0.988275,0.989950,0.990787
TN,1.0,1.0,512.0,0.0,1.0,0.989112,0.991625,0.883305,0.945422,0.971179,0.556114,0.991625,0.989112


In [134]:
df[df["align"]==1][["name"]+eval_cols].groupby(["name"]).agg(get_ci)

,NINO_ds_acc,NIFO_ds_acc,NI_h2h,NO_h2h,test_triplet_acc,test_clf_acc,test_1nn_acc,rNINO_ds_acc
name,,,,,,,,
MTL0.2,0.007207,0.025986,0.062288,0.080156,0.030150,0.025986,0.028828,0.031415
MTL0.5,0.000000,0.033027,0.067449,0.177430,0.016216,0.033027,0.024966,0.031415
MTL0.8,0.000000,0.014414,0.073302,0.111309,0.010419,0.014414,0.012483,0.007207
TN,0.031415,0.007207,0.042706,0.048093,0.025330,0.226881,0.007207,0.031415


In [133]:
df[(df["align"]==1) & (df["name"]=="MTL0.5")]

,name,model,seed,weights,filtered,embed_dim,lamda,align,wandb_project,NINO_ds_acc,NIFO_ds_acc,NI_h2h,NO_h2h,test_triplet_acc,test_clf_acc,test_1nn_acc,rNINO_ds_acc
1,MTL0.5,MTL,2,"[1, 1, 0, 0]",True,512,0.5,1.0,wv_3d_linear,0.98995,0.972362,0.850084,0.860972,0.976678,0.972362,0.987437,0.979900
9,MTL0.5,MTL,1,"[1, 1, 0, 0]",True,512,0.5,1.0,wv_3d_linear,0.98995,0.984925,0.867253,0.901591,0.971938,0.984925,0.992462,0.984925
22,MTL0.5,MTL,0,"[1, 1, 0, 0]",True,512,0.5,1.0,wv_3d_linear,0.98995,0.982412,0.876884,0.932161,0.978195,0.982412,0.982412,0.992462


In [43]:
cis = []
for align in ["0.5","0.8","0.83","0.92","0.925","1"]:
    data = df[df["group"]==f"align={align}"]["NIFO_ds_acc_k=1"]
    cis.append(np.around(get_ci(data),3))

In [112]:
df = pd.read_csv("link/wv_3d_square/wv_3d_square_num_512.csv")
MTL = df[df["wandb_name"]=="MTL0.5"]

In [113]:
# unfiltered_MTL = TN[["filtered" not in x for x in TN["wandb_group"]]]
MTL = MTL[["0.925" in x for x in MTL["wandb_group"]]]

In [118]:
cis = []
for align in [0.5,0.25,0.125,0.0625,0.03125,0.015625,0.0078125]:
    data = MTL[MTL["wandb_group"]==f"align=0.925_p={align}"]["h2h_512"]
    cis.append(np.around(get_ci(data)/2,3))

In [119]:
cis

[0.006, 0.02, 0.043, 0.016, 0.034, 0.006, 0.045]